In [1]:
![ -e python3-midi-master.zip ] || curl -LJO https://github.com/louisabraham/python3-midi/archive/refs/heads/master.zip  &> /dev/null
![ -d python3-midi-master ] || unzip python3-midi-master.zip &> /dev/null
!apt install swig &> /dev/null && cd python3-midi-master && python3 setup.py install &> /dev/null

In [2]:
import midi
import sys
from collections import defaultdict

def getData(midi_file_path):
    tracks = midi.read_midifile(midi_file_path)
    ranges = [[] for _ in range(len(tracks))]

    for evt in tracks[0]:
        if isinstance(evt, midi.SetTempoEvent):
            deltaSec = 60 / (evt.bpm * tracks.resolution) # seconds per tick
            break
    for i, track in enumerate(tracks):
        time = 0.0
        pitches = defaultdict(list)
        for evt in track:
            time += evt.tick * deltaSec
            if not isinstance(evt, midi.NoteOnEvent): continue
            if isinstance(evt, midi.NoteOffEvent) or evt.velocity == 0:
                pitches[evt.pitch][-1][1] = time # end
            else:
                pitches[evt.pitch].append([time, -1]) # start
        ranges[i] = sorted(list({tuple(arr) for vals in pitches.values() for arr in vals})) # remove duplicated keys
    return ranges

path = '/content/DJ Yoshitaka - FLOWER.mid'

try:
    data = getData(path)
except:
    print(f"Can't read file {path}.")
    exit()

try:
    with open('midi.txt', 'w') as f:
        f.write(str([[list(tup) for tup in arr] for arr in data]))
except:
    print("Can't write to midi.txt.")
print('succeed')

succeed
